In [1]:
import numpy as np
import scipy.stats as stats
import csv
import seaborn as sns
import pandas as pd
import re
%matplotlib inline

In [2]:
# Cargamos el dataset
df = pd.read_csv('properatti.csv')
print(df.columns)
df.head(1)

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...


In [3]:
df = df.drop(['Unnamed: 0','image_thumbnail', 'properati_url'], axis=1)

df.rename(columns={ 'operation': 'Tipo_Operacion',
                    'property_type': 'Tipo_Propiedad',
                    'place_name': 'Barrio',
                    'place_with_parent_names': 'Ubicacion_Completa',
                    'country_name': 'Pais',
                    'state_name': 'Provincia',
                    'geonames_id': 'GeoNames_ID',
                    'lat-lon': 'Latitud_Longitud',
                    'lat': 'Latitud',
                    'lon': 'Longitud',
                    'price': 'Precio',
                    'Currency': 'Moneda',
                    'price_aprox_local_currency': 'Precio_ARS',
                    'price_aprox_usd': 'Precio_Dolares',
                    'surface_total_in_m2': 'Metros_Totales',
                    'surface_covered_in_m2': 'Metros_Cubiertos',
                    'price_usd_per_m2': 'Precio_m2',
                    'floor': 'Pisos',
                    'rooms': 'Cantidad_Ambientes',
                    'expenses': 'Expensas',
                    'description': 'Descripcion',
                    'title': 'Titulo'}, inplace=True)

In [4]:
print(df.columns)
df.head(1)

Index(['Tipo_Operacion', 'Tipo_Propiedad', 'Barrio', 'Ubicacion_Completa',
       'Pais', 'Provincia', 'GeoNames_ID', 'Latitud_Longitud', 'Latitud',
       'Longitud', 'Precio', 'currency', 'Precio_ARS', 'Precio_Dolares',
       'Metros_Totales', 'Metros_Cubiertos', 'Precio_m2', 'price_per_m2',
       'Pisos', 'Cantidad_Ambientes', 'Expensas', 'Descripcion', 'Titulo'],
      dtype='object')


,Tipo_Operacion,Tipo_Propiedad,Barrio,Ubicacion_Completa,Pais,Provincia,GeoNames_ID,Latitud_Longitud,Latitud,Longitud,...,Precio_Dolares,Metros_Totales,Metros_Cubiertos,Precio_m2,price_per_m2,Pisos,Cantidad_Ambientes,Expensas,Descripcion,Titulo
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,62000.0,55.0,40.0,1127.272727,1550.0,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 23 columns):
Tipo_Operacion        121220 non-null object
Tipo_Propiedad        121220 non-null object
Barrio                121197 non-null object
Ubicacion_Completa    121220 non-null object
Pais                  121220 non-null object
Provincia             121220 non-null object
GeoNames_ID           102503 non-null float64
Latitud_Longitud      69670 non-null object
Latitud               69670 non-null float64
Longitud              69670 non-null float64
Precio                100810 non-null float64
currency              100809 non-null object
Precio_ARS            100810 non-null float64
Precio_Dolares        100810 non-null float64
Metros_Totales        81892 non-null float64
Metros_Cubiertos      101313 non-null float64
Precio_m2             68617 non-null float64
price_per_m2          87658 non-null float64
Pisos                 7899 non-null float64
Cantidad_Ambientes    47390 non

In [6]:
df.describe()

,GeoNames_ID,Latitud,Longitud,Precio,Precio_ARS,Precio_Dolares,Metros_Totales,Metros_Cubiertos,Precio_m2,price_per_m2,Pisos,Cantidad_Ambientes,Expensas
count,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [7]:
df.columns.value_counts()

Precio_Dolares        1
Pisos                 1
GeoNames_ID           1
Latitud_Longitud      1
Precio_m2             1
Tipo_Operacion        1
Cantidad_Ambientes    1
Expensas              1
Titulo                1
Descripcion           1
Ubicacion_Completa    1
price_per_m2          1
Longitud              1
currency              1
Provincia             1
Pais                  1
Precio_ARS            1
Tipo_Propiedad        1
Precio                1
Barrio                1
Metros_Totales        1
Metros_Cubiertos      1
Latitud               1
dtype: int64

In [8]:
#cantidad de datos valores en nulo que tiene cada columna, como para identificar cual se puede quitar
df.isnull().sum()

Tipo_Operacion             0
Tipo_Propiedad             0
Barrio                    23
Ubicacion_Completa         0
Pais                       0
Provincia                  0
GeoNames_ID            18717
Latitud_Longitud       51550
Latitud                51550
Longitud               51550
Precio                 20410
currency               20411
Precio_ARS             20410
Precio_Dolares         20410
Metros_Totales         39328
Metros_Cubiertos       19907
Precio_m2              52603
price_per_m2           33562
Pisos                 113321
Cantidad_Ambientes     73830
Expensas              106958
Descripcion                2
Titulo                     0
dtype: int64

In [9]:
#idem anterior pero en porcentaje
porcentaje_faltante = df.isnull().sum()/len(df)
porcentaje_faltante.sort_values()*100

Tipo_Operacion         0.000000
Provincia              0.000000
Pais                   0.000000
Titulo                 0.000000
Tipo_Propiedad         0.000000
Ubicacion_Completa     0.000000
Descripcion            0.001650
Barrio                 0.018974
GeoNames_ID           15.440521
Metros_Cubiertos      16.422208
Precio_ARS            16.837156
Precio_Dolares        16.837156
Precio                16.837156
currency              16.837981
price_per_m2          27.686850
Metros_Totales        32.443491
Longitud              42.525986
Latitud               42.525986
Latitud_Longitud      42.525986
Precio_m2             43.394654
Cantidad_Ambientes    60.905791
Expensas              88.234615
Pisos                 93.483749
dtype: float64

In [10]:
##df.notnull().sum()
#idem anterior pero porcentaje completo
procentaje_completo = df.notnull().sum()/len(df)
procentaje_completo.sort_values()*100

Pisos                   6.516251
Expensas               11.765385
Cantidad_Ambientes     39.094209
Precio_m2              56.605346
Longitud               57.474014
Latitud_Longitud       57.474014
Latitud                57.474014
Metros_Totales         67.556509
price_per_m2           72.313150
currency               83.162019
Precio                 83.162844
Precio_ARS             83.162844
Precio_Dolares         83.162844
Metros_Cubiertos       83.577792
GeoNames_ID            84.559479
Barrio                 99.981026
Descripcion            99.998350
Tipo_Operacion        100.000000
Pais                  100.000000
Ubicacion_Completa    100.000000
Tipo_Propiedad        100.000000
Provincia             100.000000
Titulo                100.000000
dtype: float64

In [11]:
#todos los indices que tienen los campos totalmente completos
df.dropna()

,Tipo_Operacion,Tipo_Propiedad,Barrio,Ubicacion_Completa,Pais,Provincia,GeoNames_ID,Latitud_Longitud,Latitud,Longitud,...,Precio_Dolares,Metros_Totales,Metros_Cubiertos,Precio_m2,price_per_m2,Pisos,Cantidad_Ambientes,Expensas,Descripcion,Titulo
96,sell,apartment,San Bernardo,|Argentina|Buenos Aires Costa Atlántica|San Be...,Argentina,Buenos Aires Costa Atlántica,3429228.0,"-36.6806689,-56.6774832",-36.680669,-56.677483,...,54000.0,75.0,30.0,720.000000,1800.000000,3.0,2.0,800.0,"Departamento de 2 ambientes, ubicado en la cos...",DEPARTAMENTO EN VENTA
691,sell,PH,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5666711,-58.4720061",-34.566671,-58.472006,...,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,2.0,1800.0,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA
692,sell,PH,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5666711,-58.4720061",-34.566671,-58.472006,...,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,3.0,1800.0,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA
693,sell,PH,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5666225,-58.4721291",-34.566623,-58.472129,...,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,3.0,1800.0,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA
694,sell,PH,Coghlan,|Argentina|Capital Federal|Coghlan|,Argentina,Capital Federal,3435359.0,"-34.5666643,-58.471989",-34.566664,-58.471989,...,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,2.0,1800.0,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA
695,sell,PH,Coghlan,|Argentina|Capital Federal|Coghlan|,Argentina,Capital Federal,3435359.0,"-34.5666643,-58.471989",-34.566664,-58.471989,...,152000.0,52.0,52.0,2923.076923,2923.076923,6.0,3.0,1800.0,"Venta de Departamento 1 DORM en BELGRANO, Capi...",PH EN VENTA
746,sell,apartment,Caballito,|Argentina|Capital Federal|Caballito|,Argentina,Capital Federal,3435874.0,"-34.6259473144,-58.4328219442",-34.625947,-58.432822,...,158000.0,71.0,62.0,2225.352113,2548.387097,2.0,3.0,2653.0,Venta de Departamento 3 AMBIENTES en Caballito...,DEPARTAMENTO EN VENTA
781,sell,apartment,Coghlan,|Argentina|Capital Federal|Coghlan|,Argentina,Capital Federal,3435359.0,"-34.5644671,-58.4723081",-34.564467,-58.472308,...,138000.0,66.0,56.0,2090.909091,2464.285714,7.0,2.0,2000.0,EXCELENTE DEPARTAMENTO SEMIPISO A ESTRENAR DE ...,DEPARTAMENTO EN VENTA
1648,sell,apartment,Balvanera,|Argentina|Capital Federal|Balvanera|,Argentina,Capital Federal,6693228.0,"-34.6056268,-58.392264",-34.605627,-58.392264,...,146000.0,53.0,53.0,2754.716981,2754.716981,6.0,3.0,1400.0,Venta de Departamento 3 AMBIENTES en Balvanera...,DEPARTAMENTO EN VENTA
4829,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.5802991,-58.4355947",-34.580299,-58.435595,...,480000.0,188.0,143.0,2553.191489,3356.643357,5.0,5.0,8000.0,Excelente oportunidad de compra de departament...,DEPARTAMENTO EN VENTA


In [12]:
#cantidad de porpiedades por sus tipos
df["Tipo_Propiedad"].value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: Tipo_Propiedad, dtype: int64

In [13]:
division_Ubicacion=df["Ubicacion_Completa"].str.split("\|",expand=True)

In [14]:
print(division_Ubicacion)

       0           1                             2                  3  \
0          Argentina               Capital Federal          Mataderos   
1          Argentina        Bs.As. G.B.A. Zona Sur           La Plata   
2          Argentina               Capital Federal          Mataderos   
3          Argentina               Capital Federal            Liniers   
4          Argentina  Buenos Aires Costa Atlántica      Mar del Plata   
5          Argentina                    Entre Ríos       Gualeguaychú   
6          Argentina      Bs.As. G.B.A. Zona Norte      Vicente López   
7          Argentina               Capital Federal           Belgrano   
8          Argentina               Capital Federal           Belgrano   
9          Argentina                      Santa Fe            Rosario   
10         Argentina                       Córdoba            Córdoba   
11         Argentina      Bs.As. G.B.A. Zona Norte         San Miguel   
12         Argentina      Bs.As. G.B.A. Zona Norte 

In [15]:
division_Ubicacion.rename(columns={1:"Pais", 2:"Provincia",3:"Partido",4:"Localidad",5:"otro", 6:"otro2"},inplace=True)
division_Ubicacion.head()

,0,Pais,Provincia,Partido,Localidad,otro,otro2
0,,Argentina,Capital Federal,Mataderos,,None,None
1,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,None,None
2,,Argentina,Capital Federal,Mataderos,,None,None
3,,Argentina,Capital Federal,Liniers,,None,None
4,,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,,None


In [16]:
division_Ubicacion["otro"]=division_Ubicacion["otro"].apply(lambda x: np.nan if (x=="")  or (x==None) else x)
division_Ubicacion["otro2"]=division_Ubicacion["otro2"].apply(lambda x: np.nan if (x=="")  or (x==None) else x)
division_Ubicacion["Partido"]=division_Ubicacion["Partido"].apply(lambda x: np.nan if (x=="")  or (x==None) else x)
division_Ubicacion["Localidad"]=division_Ubicacion["Localidad"].apply(lambda x: np.nan if (x=="") or (x==None) else x)

In [17]:
division_Ubicacion

,0,Pais,Provincia,Partido,Localidad,otro,otro2
0,,Argentina,Capital Federal,Mataderos,NaN,NaN,NaN
1,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,NaN,NaN,NaN
2,,Argentina,Capital Federal,Mataderos,NaN,NaN,NaN
3,,Argentina,Capital Federal,Liniers,NaN,NaN,NaN
4,,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,NaN,NaN
5,,Argentina,Entre Ríos,Gualeguaychú,NaN,NaN,NaN
6,,Argentina,Bs.As. G.B.A. Zona Norte,Vicente López,Munro,NaN,NaN
7,,Argentina,Capital Federal,Belgrano,NaN,NaN,NaN
8,,Argentina,Capital Federal,Belgrano,NaN,NaN,NaN
9,,Argentina,Santa Fe,Rosario,NaN,NaN,NaN


In [18]:
## incompletos
porcentaje_faltante_div = division_Ubicacion.isnull().sum()/len(df)
porcentaje_faltante_div.sort_values()*100

0              0.000000
Pais           0.000000
Provincia      0.000000
Partido        3.943244
Localidad     66.658142
otro          99.547929
otro2        100.000000
dtype: float64

In [19]:
## elimino los que tiene gran porcentaje vacio
division_Ubicacion = division_Ubicacion.drop(['otro', 'otro2'], axis=1)
division_Ubicacion.head()

,0,Pais,Provincia,Partido,Localidad
0,,Argentina,Capital Federal,Mataderos,NaN
1,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,NaN
2,,Argentina,Capital Federal,Mataderos,NaN
3,,Argentina,Capital Federal,Liniers,NaN
4,,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro


In [20]:
df=pd.concat([df,division_Ubicacion],axis=1)

In [21]:
df.head()

,Tipo_Operacion,Tipo_Propiedad,Barrio,Ubicacion_Completa,Pais,Provincia,GeoNames_ID,Latitud_Longitud,Latitud,Longitud,...,Pisos,Cantidad_Ambientes,Expensas,Descripcion,Titulo,0,Pais,Provincia,Partido,Localidad
0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,NaN,NaN,NaN,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,,Argentina,Capital Federal,Mataderos,NaN
1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,NaN,NaN,NaN,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,NaN
2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,NaN,NaN,NaN,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,,Argentina,Capital Federal,Mataderos,NaN
3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,NaN,NaN,NaN,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,,Argentina,Capital Federal,Liniers,NaN
4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,NaN,NaN,NaN,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro


Ambientes

In [22]:
# cantidades por ambientes
df["Cantidad_Ambientes"].value_counts()

3.0     12567
2.0     12433
4.0      7937
1.0      7377
5.0      3782
6.0      1403
7.0       770
8.0       409
10.0      226
9.0       184
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
24.0        1
27.0        1
29.0        1
31.0        1
28.0        1
Name: Cantidad_Ambientes, dtype: int64

In [23]:
#Cantidad de ambientes vacios
ambientes_vacios = df.apply(lambda row: pd.isnull(row['Cantidad_Ambientes']) , axis = 1)
len([t for t in ambientes_vacios if t==True])

73830

In [24]:

def ambientes_descripcion(row):
    if pd.isnull(row['Cantidad_Ambientes']) & pd.notnull(row['Descripcion']):
        num = re.findall("\d amb", row['Descripcion'],re.IGNORECASE)
        if len(num) > 0:
            return int(num[0].split(" ")[0])
        else:
            num = re.findall("\d dorm", row['Descripcion'],re.IGNORECASE)
            if len(num) > 0:
                return int(num[0].split(" ")[0])+1
            else:
                num = re.findall("\d habi", row['Descripcion'],re.IGNORECASE)
                if len(num) > 0:
                    return int(num[0].split(" ")[0])+1
    return row['Cantidad_Ambientes']

In [25]:
ambientes = df.apply(ambientes_descripcion, axis=1)

In [26]:
df['Cantidad_Ambientes'] = ambientes

In [27]:
# cantidades por ambientes
df["Cantidad_Ambientes"].value_counts()

3.0     26711
2.0     22401
4.0     18342
1.0      8471
5.0      6806
6.0      2074
7.0       952
8.0       448
10.0      237
9.0       217
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
0.0        25
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
31.0        1
27.0        1
24.0        1
28.0        1
29.0        1
Name: Cantidad_Ambientes, dtype: int64

# Codigo de Ramiro

# Parseo en la descripcion la RegEx que necesito, y le apendo la columna al df
import re
patron = r'(\d+)(\s)(\b(ambientes|ambiente|amb|cuarto|cuartos|habitacion|habitaciones|dormitorio|dormitorios)\b)'

ambientes = df.Descripcion.str.extract(patron, flags= re.IGNORECASE, expand = False)[0]

df['Cantidad_Ambientes'] = ambientes

In [28]:
#Cantidad de ambientes vacios que quedaron despues de correr la funcion
ambientes_vacios = df.apply(lambda row: pd.isnull(row['Cantidad_Ambientes']) , axis = 1)
len([t for t in ambientes_vacios if t==True])

34234

In [29]:
# cantidades por ambientes
df["Cantidad_Ambientes"].value_counts()

3.0     26711
2.0     22401
4.0     18342
1.0      8471
5.0      6806
6.0      2074
7.0       952
8.0       448
10.0      237
9.0       217
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
0.0        25
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
31.0        1
27.0        1
24.0        1
28.0        1
29.0        1
Name: Cantidad_Ambientes, dtype: int64

In [30]:
## calcular maximo y minimo de metros cuadrados por ambiente de ambientes
df.groupby('Cantidad_Ambientes')["Metros_Totales"].aggregate([min, max ])

,min,max
Cantidad_Ambientes,,
0.0,30.0,1568.0
1.0,0.0,49800.0
2.0,0.0,16717.0
3.0,0.0,200000.0
4.0,0.0,154438.0
5.0,0.0,100350.0
6.0,0.0,24000.0
7.0,32.0,14000.0
8.0,72.0,5000.0


In [31]:
def calcular_ambientes_m2(row):
    if pd.isnull(row['Cantidad_Ambientes'] ) & pd.notnull(row['Metros_Totales'] ): # or pd.to_numeric(row['Cantidad_Ambientes'] > 7 ) > 7
        totalm2 = int(row['Metros_Totales'])
        if totalm2 <= 35:
            return 1
        elif 35 < totalm2 <= 50:
            return 2
        elif 55 < totalm2 <= 100:
            return 3
        elif 100 < totalm2 <= 150:
            return 4
        elif 150 < totalm2 <= 200:
            return 5
        else:
            return 6
    return row['Cantidad_Ambientes']

In [32]:
ambientes = df.apply(calcular_ambientes_m2 ,axis =1)


In [33]:
df['Cantidad_Ambientes'] = ambientes

In [34]:
df["Cantidad_Ambientes"].value_counts()

3.0     31869
2.0     27011
4.0     20737
1.0     10945
6.0      9535
5.0      8397
7.0       952
8.0       448
10.0      237
9.0       217
11.0       72
12.0       65
13.0       33
14.0       27
15.0       26
0.0        25
17.0       16
16.0       11
20.0       11
22.0        8
18.0        6
21.0        5
30.0        4
25.0        4
19.0        4
32.0        3
23.0        2
31.0        1
27.0        1
24.0        1
28.0        1
29.0        1
Name: Cantidad_Ambientes, dtype: int64

In [35]:
#Cantidad de ambientes vacios que quedaron despues de correr la funcion
ambientes_vacios = df.apply(lambda row: pd.isnull(row['Cantidad_Ambientes']) , axis = 1)
len([t for t in ambientes_vacios if t==True])

10545